In [1]:
from pathlib import Path

import airline
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import pytorch_lightning as pl
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch.utils.data import DataLoader, TensorDataset

pio.templates.default = "plotly_white"

2023-02-27 11:28:39.560511: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 11:28:40.607833: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-27 11:28:40.607942: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-27 11:28:40.607951: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# %reload_ext autoreload
# %autoreload 2

In [3]:
# load the dataset
filepath = Path("airline_passenger.txt")
air = pd.read_csv(
  filepath,
  parse_dates=["month"],
  names=["month", "passengers"],
  index_col="month",
  header=0,
  dtype={"passengers": "float32"},
)
air


,passengers
month,
1949-01-01,112.0
1949-02-01,118.0
1949-03-01,132.0
1949-04-01,129.0
1949-05-01,121.0
...,...
1960-08-01,606.0
1960-09-01,508.0
1960-10-01,461.0


In [4]:
# split the dataset into train and test
test_size = 1 / 3
train, test = train_test_split(air, test_size=test_size, shuffle=False)

In [5]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train.passengers, name="train"))
fig.add_trace(go.Scatter(x=test.index, y=test.passengers, name="test"))
fig.update_layout(title="Airline passengers")
fig.show()


In [6]:
train.head()

,passengers
month,
1949-01-01,112.0
1949-02-01,118.0
1949-03-01,132.0
1949-04-01,129.0
1949-05-01,121.0


In [7]:
SEQ_LENGTH = 1
N_EPOCHS = 200
BATCH_SIZE = 16

train_sequences = airline.create_sequences(data=train, seq_length=SEQ_LENGTH)
test_sequences = airline.create_sequences(data=test, seq_length=SEQ_LENGTH)

data_module = airline.AirlineDataModule(
  train_sequences=train_sequences, test_sequences=test_sequences, batch_size=BATCH_SIZE
)
data_module.setup()

train_dataset = airline.AirlineDataset(sequences=train_sequences)
for item in train_dataset:
  print(item["sequence"].shape)
  print(item["label"].shape)
  print(item["label"])
  break


torch.Size([1, 1])
torch.Size([1])
tensor([118.])


## Model

In [8]:
model = airline.PassengerPredictor(n_features=1, lr=0.1, weight_decay=0.0)
model

PassengerPredictor(
  (model): PassengerPredictionModel(
    (lstm): LSTM(1, 20, num_layers=2, batch_first=True)
    (linear): Linear(in_features=20, out_features=1, bias=True)
  )
  (criterion): MSELoss()
)

In [9]:
for item in data_module.train_dataloader():
  print(item["sequence"].shape)
  print(item["label"].shape)
  break

torch.Size([16, 1, 1])
torch.Size([16, 1])


In [10]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
  monitor="val_loss",
  dirpath="checkpoints",
  save_top_k=1,
  mode="min",
  filename="best_checkpoint",
  verbose=True,
)
logger = pl.loggers.TensorBoardLogger("logs", name="airline")

early_stop_callback = pl.callbacks.EarlyStopping("val_loss", patience=2)
trainer = pl.Trainer(
  logger=logger,
  callbacks=[checkpoint_callback, early_stop_callback],
  max_epochs=N_EPOCHS,
  gpus=1,
)


/media/joris/apps/miniconda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:474: LightningDeprecationWarning:

Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
trainer.fit(model, data_module)

/media/joris/apps/miniconda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning:

Checkpoint directory /media/joris/uca-msc-dsai/dl_computer_vision/assignments/1/checkpoints exists and is not empty.

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                     | Params
-------------------------------------------------------
0 | model     | PassengerPredictionModel | 5.2 K 
1 | criterion | MSELoss                  | 0     
-------------------------------------------------------
5.2 K     Trainable params
0         Non-trainable params
5.2 K     Total params
0.021     Total estimated model params size (MB)
